In [162]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, mutual_info_classif
from sklearn.model_selection import StratifiedKFold, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix, log_loss
from scipy import stats
from imblearn.under_sampling import RandomUnderSampler
import warnings
warnings.filterwarnings('ignore')

In [163]:
# Load data
train_df = pd.read_csv("P2_data_stroke_train.csv")
test_df = pd.read_csv("P2_data_stroke_test.csv")
train_df

,Participant ID,Sex,Age,Race,Income,Edu,Systolic,Diastolic,Pulse,BMI,...,Trig,LDL,TCHOL,eGFR,A1C,uACR,CurrentSmoker,Diabetes,Insurance,stroke
0,1,2,47,6,3.73,4,127.666667,77.000000,52.666667,25.9,...,63.0,83.0,167.0,90.313080,5.4,5.49,2,2,Private,2
1,2,1,49,4,0.95,5,90.000000,52.333333,97.333333,39.9,...,141.0,134.0,193.0,58.109773,6.0,52.42,1,1,Medicare,2
2,3,1,63,3,0.30,5,124.666667,86.333333,118.333333,32.9,...,NaN,NaN,230.0,97.294339,5.6,28.87,1,2,Uninsured,2
3,4,1,64,4,1.73,3,130.000000,74.333333,NaN,34.1,...,NaN,NaN,168.0,71.817296,5.2,5.17,2,2,Uninsured,2
4,5,2,49,4,1.54,3,130.666667,82.000000,74.333333,23.9,...,NaN,NaN,215.0,85.342018,5.4,18.20,2,2,Uninsured,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13694,13695,1,68,4,NaN,4,149.000000,76.333333,82.666667,NaN,...,282.0,102.0,174.0,47.350256,9.7,2505.26,2,1,Medicare,2
13695,13696,1,72,7,NaN,4,138.000000,75.666667,62.333333,30.1,...,56.0,123.0,178.0,95.880112,5.1,34.23,2,2,Medicare,1
13696,13697,2,21,4,NaN,4,NaN,NaN,NaN,22.8,...,NaN,NaN,145.0,125.046973,5.4,5.64,2,2,Medicaid,2
13697,13698,2,39,1,2.64,4,133.000000,89.666667,67.333333,31.5,...,NaN,NaN,138.0,97.848275,5.3,11.03,2,2,Private,2


In [164]:
test_df

,Participant ID,Sex,Age,Race,Income,Edu,Systolic,Diastolic,Pulse,BMI,...,Trig,LDL,TCHOL,eGFR,A1C,uACR,CurrentSmoker,Diabetes,Insurance,stroke
0,101,2,77,1,2.63,3,96.666667,16.000000,76.000000,25.1,...,NaN,NaN,230.0,37.963433,7.3,249.65,2,1,Medicare,NaN
1,102,1,69,7,0.80,4,140.000000,63.333333,64.000000,27.9,...,51.0,136.0,214.0,94.741111,5.7,72.26,2,2,Medicaid,NaN
2,103,1,52,4,0.94,3,125.333333,70.000000,62.000000,25.4,...,NaN,NaN,NaN,NaN,NaN,1.20,2,2,Private,NaN
3,104,2,37,2,0.58,1,112.000000,66.666667,82.000000,27.7,...,NaN,NaN,153.0,121.188912,5.9,0.89,2,2,Uninsured,NaN
4,105,1,78,3,1.88,3,123.333333,65.333333,74.000000,26.4,...,NaN,NaN,139.0,72.656355,6.0,8.77,2,2,Medicare,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,396,2,54,3,5.00,5,128.000000,76.666667,88.000000,30.6,...,177.0,150.0,231.0,103.321438,5.7,5.71,2,2,Private,NaN
296,397,2,76,6,5.00,4,121.333333,70.000000,69.000000,20.7,...,NaN,NaN,173.0,82.804007,7.2,7.97,2,1,Medicare,NaN
297,398,2,31,4,0.54,3,100.666667,64.000000,72.000000,20.2,...,47.0,121.0,177.0,104.444575,5.6,5.79,1,2,Medicaid,NaN
298,399,1,28,4,5.00,5,136.000000,74.000000,52.666667,31.1,...,NaN,NaN,160.0,95.861150,4.8,9.83,2,2,Private,NaN


In [165]:
# drop specified columns
train_df = train_df.drop(columns = ['Insurance', 'Income', 'Edu'], errors = 'ignore')
test_df = test_df.drop(columns = ['Insurance', 'Income', 'Edu'], errors = 'ignore')


In [166]:
# Drop rows with no predictions
train_df = train_df.dropna(subset=['stroke'])


In [167]:
# Fix class labels (2 -> 0, 1 -> 1)
train_df['stroke'] = (train_df['stroke'] == 1).astype(int)
train_df['stroke']

0        0
1        0
2        0
3        0
4        0
        ..
13694    0
13695    1
13696    0
13697    0
13698    0
Name: stroke, Length: 13699, dtype: int64

In [168]:
# Count distribution
print(f"Class distribution after fixing labels:")
print(train_df['stroke'].value_counts())

Class distribution after fixing labels:
stroke
0    13158
1      541
Name: count, dtype: int64


In [169]:
def create_stroke_features(df):
    data = df.copy()

    # Cardiovascular ratios
    data['BP_Difference'] = data['Systolic'] - data['Diastolic']
    data['Cholesterol_Ratio'] = data['TCHOL'] / (data['HDL'] + 1e-8)
    data['LDL_HDL_Ratio'] = data['LDL'] / (data['HDL'] + 1e-8)
    data['Non_HDL'] = data['TCHOL'] - data['HDL']

    # Age interactions
    data['Age_Systolic_Interaction'] = data['Age'] * data['Systolic']
    data['Age_BMI_Interaction'] = data['Age'] * data['BMI']
    data['Age_Tchol_Interaction'] = data['Age'] * data['TCHOL']
    data['Age_Diabetes_Interaction'] = data['Age'] * data['Diabetes']
    data['Age_Smoking_Interaction'] = data['Age'] * data['CurrentSmoker']
    data['Age_Age'] = data['Age'] * data['Age']

    # Risk factor interactions
    data['Diabetes_Smoking_Interaction'] = data['Diabetes'] * data['CurrentSmoker']
    data['BMI_Diabetes_Interaction'] = data['BMI'] * data['Diabetes']

    # Categories
    data['Age_Senior'] = (data['Age'] >= 65).astype(int)
    data['BMI_Obese'] = (data['BMI'] >= 30).astype(int)
    data['BP_High'] = (data['Systolic'] >= 140).astype(int)
    data['BP_Crisis'] = (data['Systolic'] >= 180).astype(int)

    # Cholesterol categories 
    data['TCHOL_High'] = (data['TCHOL'] >= 200).astype(int)
    data['HDL_Low'] = (data['HDL'] < 40).astype(int)
    data['LDL_High'] = (data['LDL'] >= 160).astype(int)

    # Lipid ratio categories
    data['Cholesterol_Ratio_High'] = (data['Cholesterol_Ratio'] >= 5).astype(int)
    data['LDL_HDL_Ratio_High'] = (data['LDL_HDL_Ratio'] >= 3).astype(int)

    # Kidney function categories
    data['eGFR_Low'] = (data['eGFR'] < 60).astype(int)
    data['uACR_High'] = (data['uACR'] >= 30).astype(int)

    # Diabetes / A1C categories
    data['A1C_High'] = (data['A1C'] >= 6.5).astype(int)

    # Hemodynamics
    data['Mean_Arterial_Pressure'] = (data['Systolic'] + 2.0 * data['Diastolic']) / 3.0
    data['Pulse_Pressure_to_Age'] = data['BP_Difference'] / (data['Age'] + 1e-6)

    # Lifestyle/metabolic interactions
    data['BMI_Smoking_Interaction'] = data['BMI'] * data['CurrentSmoker']
    data['BP_Smoking_Interaction'] = data['Systolic'] * data['CurrentSmoker']
    data['BMI_to_Age'] = data['BMI'] / (data['Age'] + 1e-6)

    # Renal stress proxy (microvascular damage signal)
    data['Renal_Stress_Index'] = data['uACR'] / (data['eGFR'] + 1e-6)

    # inflammation proxy 
    if 'CRP' in data.columns:
        data['CRP_High'] = (data['CRP'] >= 3).astype(int) 

    return data


In [170]:
# Apply feature engineering
train_df_enhanced = create_stroke_features(train_df)
test_df_enhanced = create_stroke_features(test_df)

X = train_df_enhanced.drop(columns=['stroke'])
y = train_df_enhanced['stroke']



In [171]:
X

,Participant ID,Sex,Age,Race,Systolic,Diastolic,Pulse,BMI,HDL,Trig,...,LDL_HDL_Ratio_High,eGFR_Low,uACR_High,A1C_High,Mean_Arterial_Pressure,Pulse_Pressure_to_Age,BMI_Smoking_Interaction,BP_Smoking_Interaction,BMI_to_Age,Renal_Stress_Index
0,1,2,47,6,127.666667,77.000000,52.666667,25.9,70.0,63.0,...,0,0,0,0,93.888889,1.078014,51.8,255.333333,0.551064,0.060789
1,2,1,49,4,90.000000,52.333333,97.333333,39.9,32.0,141.0,...,1,1,1,0,64.888889,0.768707,39.9,90.000000,0.814286,0.902086
2,3,1,63,3,124.666667,86.333333,118.333333,32.9,44.0,NaN,...,0,0,0,0,99.111111,0.608466,32.9,124.666667,0.522222,0.296728
3,4,1,64,4,130.000000,74.333333,NaN,34.1,49.0,NaN,...,0,0,0,0,92.888889,0.869792,68.2,260.000000,0.532812,0.071988
4,5,2,49,4,130.666667,82.000000,74.333333,23.9,78.0,NaN,...,0,0,0,0,98.222222,0.993197,47.8,261.333333,0.487755,0.213260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13694,13695,1,68,4,149.000000,76.333333,82.666667,NaN,32.0,282.0,...,1,1,1,1,100.555556,1.068627,NaN,298.000000,NaN,52.909110
13695,13696,1,72,7,138.000000,75.666667,62.333333,30.1,41.0,56.0,...,0,0,1,0,96.444444,0.865741,60.2,276.000000,0.418056,0.357008
13696,13697,2,21,4,NaN,NaN,NaN,22.8,69.0,NaN,...,0,0,0,0,NaN,NaN,45.6,NaN,1.085714,0.045103
13697,13698,2,39,1,133.000000,89.666667,67.333333,31.5,45.0,NaN,...,0,0,0,0,104.111111,1.111111,63.0,266.000000,0.807692,0.112726


In [172]:
# Create a 100/100 balanced test split from the train.csv file 
import numpy as np

def create_fixed_balanced_test_split(X, y, n_per_class=100, random_state=42, pos_label=1):

    # 1. Setup Random State and Convert Target to Series
    rng = np.random.RandomState(random_state)
    y_series = y if hasattr(y, "iloc") else pd.Series(y)
    y_array = y_series.to_numpy()

    # 2. Identify Indices for Each Class
    stroke_idx = np.where(y_array == pos_label)[0]
    nostroke_idx = np.where(y_array != pos_label)[0]

    # 3. Check for Sample Availability
    n_each = min(n_per_class, len(stroke_idx), len(nostroke_idx))
    if n_each < n_per_class:
        print(f"Warning: not enough samples for both classes; using n_per_class={n_each} for the test set.")

    # 4. Select Test Set Indices
    # Randomly sample 'n_each' indices from the positive and negative class pools.
    test_stroke_idx = rng.choice(stroke_idx, size=n_each, replace=False)
    test_nostroke_idx = rng.choice(nostroke_idx, size=n_each, replace=False)

    # Combine and shuffle the test indices
    test_idx = np.concatenate([test_stroke_idx, test_nostroke_idx])
    rng.shuffle(test_idx) # Optional: shuffling ensures the test set isn't ordered by class

    # 5. Define Training Set Indices
    all_idx = np.arange(len(y_array))
    train_mask = np.ones(len(y_array), dtype=bool)
    train_mask[test_idx] = False
    train_idx = all_idx[train_mask]

    # 6. Split Data
    # Use the calculated indices to slice the feature matrix (X) and labels (y_series)
    if hasattr(X, "iloc"):
        X_train_fixed, X_test_fixed = X.iloc[train_idx], X.iloc[test_idx]
    else:
        # If X is a numpy array, use direct slicing
        X_train_fixed, X_test_fixed = X[train_idx], X[test_idx]

    y_train_fixed, y_test_fixed = y_series.iloc[train_idx], y_series.iloc[test_idx]

    # 7. Print Summary and Return
    print("Fixed balanced test split created:")
    print(f"  Total Samples: {len(X)}")
    print(f"  Train Size: {len(train_idx)} | Test Size: {len(test_idx)} (exactly {2*n_each})")
    print(f"  Test Positive Class Count: {y_test_fixed.sum()} | Test Negative Class Count: {len(y_test_fixed) - y_test_fixed.sum()}")
    print(f"  Test Balance Rate: {y_test_fixed.mean()*100:.2f}% (should be 50.00%)")

    return X_train_fixed, X_test_fixed, y_train_fixed, y_test_fixed


# Use the function to create a 100/100 balanced test split
# In this case, since only 100 positive samples exist, n_each will be 100.
X_train_fixed, X_test_fixed, y_train_fixed, y_test_fixed = create_fixed_balanced_test_split(
    X, y, n_per_class=100, random_state=42
)


print("\nTraining split class distribution:")
print(y_train_fixed.value_counts())
print("\nTest split class distribution:")
print(y_test_fixed.value_counts())


Fixed balanced test split created:
  Total Samples: 13699
  Train Size: 13499 | Test Size: 200 (exactly 200)
  Test Positive Class Count: 100 | Test Negative Class Count: 100
  Test Balance Rate: 50.00% (should be 50.00%)

Training split class distribution:
stroke
0    13058
1      441
Name: count, dtype: int64

Test split class distribution:
stroke
0    100
1    100
Name: count, dtype: int64


In [173]:
X_train_fixed.columns

Index(['Participant ID', 'Sex', 'Age', 'Race', 'Systolic', 'Diastolic',
       'Pulse', 'BMI', 'HDL', 'Trig', 'LDL', 'TCHOL', 'eGFR', 'A1C', 'uACR',
       'CurrentSmoker', 'Diabetes', 'BP_Difference', 'Cholesterol_Ratio',
       'LDL_HDL_Ratio', 'Non_HDL', 'Age_Systolic_Interaction',
       'Age_BMI_Interaction', 'Age_Tchol_Interaction',
       'Age_Diabetes_Interaction', 'Age_Smoking_Interaction', 'Age_Age',
       'Diabetes_Smoking_Interaction', 'BMI_Diabetes_Interaction',
       'Age_Senior', 'BMI_Obese', 'BP_High', 'BP_Crisis', 'TCHOL_High',
       'HDL_Low', 'LDL_High', 'Cholesterol_Ratio_High', 'LDL_HDL_Ratio_High',
       'eGFR_Low', 'uACR_High', 'A1C_High', 'Mean_Arterial_Pressure',
       'Pulse_Pressure_to_Age', 'BMI_Smoking_Interaction',
       'BP_Smoking_Interaction', 'BMI_to_Age', 'Renal_Stress_Index'],
      dtype='object')

In [174]:
X_train_fixed

,Participant ID,Sex,Age,Race,Systolic,Diastolic,Pulse,BMI,HDL,Trig,...,LDL_HDL_Ratio_High,eGFR_Low,uACR_High,A1C_High,Mean_Arterial_Pressure,Pulse_Pressure_to_Age,BMI_Smoking_Interaction,BP_Smoking_Interaction,BMI_to_Age,Renal_Stress_Index
0,1,2,47,6,127.666667,77.000000,52.666667,25.9,70.0,63.0,...,0,0,0,0,93.888889,1.078014,51.8,255.333333,0.551064,0.060789
1,2,1,49,4,90.000000,52.333333,97.333333,39.9,32.0,141.0,...,1,1,1,0,64.888889,0.768707,39.9,90.000000,0.814286,0.902086
2,3,1,63,3,124.666667,86.333333,118.333333,32.9,44.0,NaN,...,0,0,0,0,99.111111,0.608466,32.9,124.666667,0.522222,0.296728
3,4,1,64,4,130.000000,74.333333,NaN,34.1,49.0,NaN,...,0,0,0,0,92.888889,0.869792,68.2,260.000000,0.532812,0.071988
4,5,2,49,4,130.666667,82.000000,74.333333,23.9,78.0,NaN,...,0,0,0,0,98.222222,0.993197,47.8,261.333333,0.487755,0.213260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13694,13695,1,68,4,149.000000,76.333333,82.666667,NaN,32.0,282.0,...,1,1,1,1,100.555556,1.068627,NaN,298.000000,NaN,52.909110
13695,13696,1,72,7,138.000000,75.666667,62.333333,30.1,41.0,56.0,...,0,0,1,0,96.444444,0.865741,60.2,276.000000,0.418056,0.357008
13696,13697,2,21,4,NaN,NaN,NaN,22.8,69.0,NaN,...,0,0,0,0,NaN,NaN,45.6,NaN,1.085714,0.045103
13697,13698,2,39,1,133.000000,89.666667,67.333333,31.5,45.0,NaN,...,0,0,0,0,104.111111,1.111111,63.0,266.000000,0.807692,0.112726


In [175]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler
from imblearn.pipeline import Pipeline 


# Pipeline
lr_pipeline = Pipeline([
    ("Missing", KNNImputer(n_neighbors=8)),
    ("feature_selector", SelectKBest(score_func=f_classif, k=17)),
    ("scaler", StandardScaler()),
    ("sampling", SMOTE(random_state=42, sampling_strategy= 'not majority')),
    ("classifier", LogisticRegression(
        random_state=42,
        class_weight='balanced',
        max_iter=2000,
        solver='liblinear',
        C=0.7743,
    ))
     ])

# Train the pipeline on the training split
print("Training advanced stroke prediction model (validation split)…")
lr_pipeline.fit(X_train_fixed, y_train_fixed)

best_model = lr_pipeline
print("Done.")

Training advanced stroke prediction model (validation split)…
Done.


In [176]:

# Selected features (by original column name)
selected_mask = lr_pipeline.named_steps['feature_selector'].get_support()
selected_features = X_train_fixed.columns[selected_mask]
print(f"Selected {len(selected_features)} most important features:")
print(f"  {list(selected_features)}")

# Predictions on the validation holdout (X_test_fixed)
y_pred       = best_model.predict(X_test_fixed)
y_pred_proba = best_model.predict_proba(X_test_fixed)[:, 1]

# Coefficient-based importance (absolute value)
importances = abs(best_model.named_steps['classifier'].coef_[0])
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'importance': importances
}).sort_values('importance', ascending=False)

print("\nTop 5 Most Important Features:")
for i, (_, row) in enumerate(feature_importance.head().iterrows(), 1):
    print(f"  {i}. {row['feature']}: {row['importance']:.4f}")

print("\nPipeline ready for stroke prediction!")
print(f"  Model trained on {len(X_train_fixed)} samples")
print(f"  Tested (validation) on {len(X_test_fixed)} samples")


Selected 17 most important features:
  ['Age', 'Systolic', 'eGFR', 'Diabetes', 'BP_Difference', 'Age_Systolic_Interaction', 'Age_BMI_Interaction', 'Age_Tchol_Interaction', 'Age_Diabetes_Interaction', 'Age_Smoking_Interaction', 'Age_Age', 'Diabetes_Smoking_Interaction', 'Age_Senior', 'BP_High', 'eGFR_Low', 'uACR_High', 'BMI_to_Age']

Top 5 Most Important Features:
  1. Age: 3.3950
  2. Age_Age: 2.1974
  3. Diabetes_Smoking_Interaction: 0.9355
  4. BMI_to_Age: 0.6912
  5. Age_Tchol_Interaction: 0.5475

Pipeline ready for stroke prediction!
  Model trained on 13499 samples
  Tested (validation) on 200 samples


In [210]:

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix, balanced_accuracy_score
from scipy import stats


def evaluate_split(name, y_true, y_pred, y_proba):
    acc = accuracy_score(y_true, y_pred)

    prec_arr, rec_arr, f1_arr, support = precision_recall_fscore_support(
        y_true, y_pred, labels=[1, 0], average=None, zero_division=0
    )
    prec_stroke, prec_nostroke = prec_arr[0], prec_arr[1]
    rec_stroke,  rec_nostroke  = rec_arr[0],  rec_arr[1]
    f1_stroke,   f1_nostroke   = f1_arr[0],   f1_arr[1]

    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])

    # KL Divergence (as log loss / cross-entropy on probabilities)
    # y_proba must be the probability of class 1
    kl_div = log_loss(y_true, y_proba, labels=[0,1])  # average over samples

    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.4f}")
    print(f"stroke(1)   — Precision: {prec_stroke:.4f}  Recall: {rec_stroke:.4f}  F1: {f1_stroke:.4f}")
    print(f"no-stroke(0) — Precision: {prec_nostroke:.4f}  Recall: {rec_nostroke:.4f}  F1: {f1_nostroke:.4f}")
    print("Confusion matrix (rows=[stroke, no-stroke], cols=[stroke, no-stroke]):\n", cm)
    print(f"KL Divergence (log loss): {kl_div:.4f}")
    print("Classification report:\n", classification_report(y_true, y_pred, digits=4, zero_division=0))

    return {"accuracy": acc, "kl_div": kl_div, "cm": cm}

# Evaluate on the validation split
y_pred = lr_pipeline.predict(X_test_fixed)
y_proba = lr_pipeline.predict_proba(X_test_fixed)[:, 1]
metrics_rf = evaluate_split("Logistic Regression", y_test_fixed, y_pred, y_proba)



=== Logistic Regression ===
Accuracy: 0.7050
stroke(1)   — Precision: 0.6952  Recall: 0.7300  F1: 0.7122
no-stroke(0) — Precision: 0.7158  Recall: 0.6800  F1: 0.6974
Confusion matrix (rows=[stroke, no-stroke], cols=[stroke, no-stroke]):
 [[73 27]
 [32 68]]
KL Divergence (log loss): 0.5930
Classification report:
               precision    recall  f1-score   support

           0     0.7158    0.6800    0.6974       100
           1     0.6952    0.7300    0.7122       100

    accuracy                         0.7050       200
   macro avg     0.7055    0.7050    0.7048       200
weighted avg     0.7055    0.7050    0.7048       200



In [200]:
final_model = lr_pipeline

# Load test + require a real participant ID column
test_df = pd.read_csv("P2_data_stroke_test.csv")
id_col = None
for cand in ["Participant ID"]:
    if cand in test_df.columns:
        id_col = cand
        break
if id_col is None:
    raise ValueError("No participant ID column found in test dataset.")

# Align columns to training features
X_test_real = test_df.reindex(columns=X_train_fixed.columns, fill_value=np.nan).copy()

# Predict probabilities on REAL test file
test_proba = final_model.predict_proba(X_test_real)[:, 1]

# Save single CSV (IDs + 4dp probs), no index
submission_df = pd.DataFrame({
    id_col: test_df[id_col],
    "logreg_probability": np.round(test_proba, 4)
})
submission_df.to_csv("stroke_predictions_logreg.csv", index=False, float_format="%.4f")
print("Saved: stroke_predictions_logreg.csv")

pred_stroke = int(test_pred.sum()) 
pred_nostroke = int(len(test_pred) - pred_stroke) 
pred_rate = pred_stroke / len(test_pred) 
print(f"\n=== P2_data_stroke_test.csv Prediction Distribution ===") 
print(f"Total: {len(test_pred)} | stroke: {pred_stroke} | no-stroke: {pred_nostroke} | stroke rate: {pred_rate:.4f}")

Saved: stroke_predictions_logreg.csv

=== P2_data_stroke_test.csv Prediction Distribution ===
Total: 300 | stroke: 149 | no-stroke: 151 | stroke rate: 0.4967


In [179]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier

rf_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("feature_selector", SelectKBest(score_func=mutual_info_classif, k=15)),  # MI handles non-linearity
    ("clf", BalancedRandomForestClassifier(
        n_estimators=800,         # more trees for stability on imbalanced data
        max_depth=None,           # let trees grow; regularize with min_samples_*
        min_samples_leaf=2,       # key regularization to reduce overfitting
        min_samples_split=4,
        max_features="sqrt",
        bootstrap=True,
        oob_score=True,           # free sanity check (won't be used for selection here)
        sampling_strategy="auto", # rebalance each tree's bootstrap
        n_jobs=-1,
        random_state=42
    ))
])




# Train the pipeline
print(" Training advanced stroke prediction model ")
rf_pipeline.fit(X_train_fixed, y_train_fixed)

best_model_rf = rf_pipeline

 Training advanced stroke prediction model 


In [180]:
# Selected features (by original column name)
selected_mask = rf_pipeline.named_steps['feature_selector'].get_support()
selected_features = X_train_fixed.columns[selected_mask]
print(f"Selected {len(selected_features)} features:")
print(f"  {list(selected_features)}")

# Predictions on the validation holdout (X_test_fixed)
y_pred       = best_model_rf.predict(X_test_fixed)
y_pred_proba = best_model_rf.predict_proba(X_test_fixed)[:, 1]

# Feature importance from RandomForest (Gini/entropy-based)
rf_importances = best_model_rf.named_steps['clf'].feature_importances_
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'importance': rf_importances
}).sort_values('importance', ascending=False)

print("\nTop 5 Most Important Features (RandomForest):")
for i, (_, row) in enumerate(feature_importance.head().iterrows(), 1):
    print(f"  {i}. {row['feature']}: {row['importance']:.4f}")

print("\nPipeline ready for stroke prediction!")
print(f"  Model trained on {len(X_train_fixed)} samples")
print(f"  Tested (validation) on {len(X_test_fixed)} samples")


Selected 15 features:
  ['Age', 'LDL', 'eGFR', 'uACR', 'Diabetes', 'Age_Systolic_Interaction', 'Age_BMI_Interaction', 'Age_Tchol_Interaction', 'Age_Diabetes_Interaction', 'Age_Smoking_Interaction', 'Age_Age', 'Age_Senior', 'eGFR_Low', 'BMI_to_Age', 'Renal_Stress_Index']

Top 5 Most Important Features (RandomForest):
  1. Age_BMI_Interaction: 0.1031
  2. eGFR: 0.1022
  3. Age_Systolic_Interaction: 0.0990
  4. Age_Age: 0.0917
  5. Age: 0.0898

Pipeline ready for stroke prediction!
  Model trained on 13499 samples
  Tested (validation) on 200 samples


In [181]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
from scipy import stats

def evaluate_split(name, y_true, y_pred, y_proba):
    acc = accuracy_score(y_true, y_pred)

    prec_arr, rec_arr, f1_arr, support = precision_recall_fscore_support(
        y_true, y_pred, labels=[1, 0], average=None, zero_division=0
    )
    prec_stroke, prec_nostroke = prec_arr[0], prec_arr[1]
    rec_stroke,  rec_nostroke  = rec_arr[0],  rec_arr[1]
    f1_stroke,   f1_nostroke   = f1_arr[0],   f1_arr[1]

    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])

    # KL Divergence (as log loss / cross-entropy on probabilities)
    # y_proba must be the probability of class 1
    kl_div = log_loss(y_true, y_proba, labels=[0,1])  # average over samples

    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.4f}")
    print(f"stroke(1)   — Precision: {prec_stroke:.4f}  Recall: {rec_stroke:.4f}  F1: {f1_stroke:.4f}")
    print(f"no-stroke(0) — Precision: {prec_nostroke:.4f}  Recall: {rec_nostroke:.4f}  F1: {f1_nostroke:.4f}")
    print("Confusion matrix (rows=[stroke, no-stroke], cols=[stroke, no-stroke]):\n", cm)
    print(f"KL Divergence (log loss): {kl_div:.4f}")
    print("Classification report:\n", classification_report(y_true, y_pred, digits=4, zero_division=0))

    return {"accuracy": acc, "kl_div": kl_div, "cm": cm}

# Evaluate on the validation split
y_pred = rf_pipeline.predict(X_test_fixed)
y_proba = rf_pipeline.predict_proba(X_test_fixed)[:, 1]
metrics_rf = evaluate_split("Random Forest", y_test_fixed, y_pred, y_proba)



=== Random Forest ===
Accuracy: 0.7250
stroke(1)   — Precision: 0.6957  Recall: 0.8000  F1: 0.7442
no-stroke(0) — Precision: 0.7647  Recall: 0.6500  F1: 0.7027
Confusion matrix (rows=[stroke, no-stroke], cols=[stroke, no-stroke]):
 [[80 20]
 [35 65]]
KL Divergence (log loss): 0.5910
Classification report:
               precision    recall  f1-score   support

           0     0.7647    0.6500    0.7027       100
           1     0.6957    0.8000    0.7442       100

    accuracy                         0.7250       200
   macro avg     0.7302    0.7250    0.7234       200
weighted avg     0.7302    0.7250    0.7234       200



In [202]:
final_model = rf_pipeline

test_df = pd.read_csv("P2_data_stroke_test.csv")
id_col = None
for cand in ["Participant ID"]:
    if cand in test_df.columns:
        id_col = cand
        break
if id_col is None:
    raise ValueError("No participant ID column found in test dataset.")

X_test_real = test_df.reindex(columns=X_train_fixed.columns, fill_value=np.nan).copy()

test_proba = final_model.predict_proba(X_test_real)[:, 1]

submission_df = pd.DataFrame({
    id_col: test_df[id_col],
    "rf_probability": np.round(test_proba, 4)
})
submission_df.to_csv("stroke_predictions_rf.csv", index=False, float_format="%.4f")
print("Saved: stroke_predictions_rf.csv")


# Optional minimal distribution info (keep if you want, otherwise comment out)
pred_stroke = int(test_pred.sum())
pred_nostroke = int(len(test_pred) - pred_stroke)
pred_rate = pred_stroke / len(test_pred)
print(f"\n=== P2_data_stroke_test.csv Prediction Distribution ===")
print(f"Total: {len(test_pred)} | stroke: {pred_stroke} | no-stroke: {pred_nostroke} | stroke rate: {pred_rate:.4f}")


Saved: stroke_predictions_rf.csv

=== P2_data_stroke_test.csv Prediction Distribution ===
Total: 300 | stroke: 149 | no-stroke: 151 | stroke rate: 0.4967


In [183]:
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

svm_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler()),
    ("undersample", RandomUnderSampler(random_state=42, sampling_strategy=0.8)),
    ("classifier", CalibratedClassifierCV(
        estimator=LinearSVC(C=0.8, class_weight=None, max_iter=5000, random_state=42),
        method="sigmoid",
        cv=3
    ))
])

print("Training SVM pipeline (validation split)…")
svm_pipeline.fit(X_train_fixed, y_train_fixed)
best_model_svm = svm_pipeline
print("Done.")


Training SVM pipeline (validation split)…
Done.


In [184]:
y_pred = svm_pipeline.predict(X_test_fixed)
y_pred_proba = svm_pipeline.predict_proba(X_test_fixed)[:, 1]


In [185]:

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
from scipy import stats
import numpy as np
import pandas as pd


def evaluate_split(name, y_true, y_pred, y_proba):
    acc = accuracy_score(y_true, y_pred)

    prec_arr, rec_arr, f1_arr, support = precision_recall_fscore_support(
        y_true, y_pred, labels=[1, 0], average=None, zero_division=0
    )
    prec_stroke, prec_nostroke = prec_arr[0], prec_arr[1]
    rec_stroke,  rec_nostroke  = rec_arr[0],  rec_arr[1]
    f1_stroke,   f1_nostroke   = f1_arr[0],   f1_arr[1]

    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])

    # KL Divergence (as log loss / cross-entropy on probabilities)
    # y_proba must be the probability of class 1
    kl_div = log_loss(y_true, y_proba, labels=[0,1])  # average over samples

    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.4f}")
    print(f"stroke(1)   — Precision: {prec_stroke:.4f}  Recall: {rec_stroke:.4f}  F1: {f1_stroke:.4f}")
    print(f"no-stroke(0) — Precision: {prec_nostroke:.4f}  Recall: {rec_nostroke:.4f}  F1: {f1_nostroke:.4f}")
    print("Confusion matrix (rows=[stroke, no-stroke], cols=[stroke, no-stroke]):\n", cm)
    print(f"KL Divergence (log loss): {kl_div:.4f}")
    print("Classification report:\n", classification_report(y_true, y_pred, digits=4, zero_division=0))

    return {"accuracy": acc, "kl_div": kl_div, "cm": cm}

# Evaluate on the validation split
y_pred = svm_pipeline.predict(X_test_fixed)
y_proba = svm_pipeline.predict_proba(X_test_fixed)[:, 1]
metrics_rf = evaluate_split("SVM", y_test_fixed, y_pred, y_proba)



=== SVM ===
Accuracy: 0.7250
stroke(1)   — Precision: 0.7473  Recall: 0.6800  F1: 0.7120
no-stroke(0) — Precision: 0.7064  Recall: 0.7700  F1: 0.7368
Confusion matrix (rows=[stroke, no-stroke], cols=[stroke, no-stroke]):
 [[68 32]
 [23 77]]
KL Divergence (log loss): 0.5959
Classification report:
               precision    recall  f1-score   support

           0     0.7064    0.7700    0.7368       100
           1     0.7473    0.6800    0.7120       100

    accuracy                         0.7250       200
   macro avg     0.7268    0.7250    0.7244       200
weighted avg     0.7268    0.7250    0.7244       200



In [204]:

final_model = svm_pipeline

test_df = pd.read_csv("P2_data_stroke_test.csv")
id_col = None
for cand in ["Participant ID"]:
    if cand in test_df.columns:
        id_col = cand
        break
if id_col is None:
    raise ValueError("No participant ID column found in test dataset.")

X_test_real = test_df.reindex(columns=X_train_fixed.columns, fill_value=np.nan).copy()

test_proba = final_model.predict_proba(X_test_real)[:, 1]

submission_df = pd.DataFrame({
    id_col: test_df[id_col],
    "svm_probability": np.round(test_proba, 4)
})
submission_df.to_csv("stroke_predictions_svm.csv", index=False, float_format="%.4f")
print("Saved: stroke_predictions_svm.csv")


pred_stroke = int(test_pred.sum())
pred_nostroke = int(len(test_pred) - pred_stroke)
pred_rate = pred_stroke / len(test_pred)
print(f"Total: {len(test_pred)} | stroke: {pred_stroke} | no-stroke: {pred_nostroke} | stroke rate: {pred_rate:.4f}")


Saved: stroke_predictions_svm.csv
Total: 300 | stroke: 149 | no-stroke: 151 | stroke rate: 0.4967


In [211]:
# Load all 3 model prediction files
logreg = pd.read_csv("stroke_predictions_logreg.csv")
rf     = pd.read_csv("stroke_predictions_rf.csv")
svm    = pd.read_csv("stroke_predictions_svm.csv")

# Detect the ID column automatically (case-insensitive)
id_col = [c for c in logreg.columns if "id" in c.lower()][0]

# Create clean 4-column final submission
final_submission = pd.DataFrame({
    "Participant ID": logreg[id_col],
    "Logistic regression prediction\np(stroke=1|X)": logreg.iloc[:, 1].round(4),
    "Random forest prediction\np(stroke=1|X)": rf.iloc[:, 1].round(4),
    "SVM prediction\np(stroke=1|X)": svm.iloc[:, 1].round(4)
})

# Save without index (only 4 columns)
final_submission.to_csv("stroke_predictions_final.csv", index=False, float_format="%.4f")
print(" Saved: stroke_predictions_final.csv")

# Display preview with no index
from IPython.display import display, HTML
display(HTML(final_submission.to_html(index=False)))


 Saved: stroke_predictions_final.csv


Participant ID,Logistic regression prediction p(stroke=1|X),Random forest prediction p(stroke=1|X),SVM prediction p(stroke=1|X)
101,0.9942,0.7347,0.6576
102,0.8401,0.5987,0.4468
103,0.3241,0.5316,0.3243
104,0.1680,0.3069,0.1948
105,0.9700,0.6499,0.3648
106,0.8859,0.5467,0.4169
107,0.8906,0.6786,0.3680
108,0.5572,0.6263,0.3399
109,0.7062,0.7227,0.4529
110,0.2122,0.3265,0.1943
